In [1]:
# Imports
import os 
import pandas as pd
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from datetime import date
from edc_appointment.models import Appointment

In [2]:
OnScheduleCohortCSec.objects.get(subject_identifier='B142-040990001-5', schedule_name='c_sec2_schedule1').delete()

OnScheduleCohortCSec.objects.get(subject_identifier='B142-040990005-6', schedule_name='c_sec2_schedule1').delete()

OnScheduleCohortCSec.objects.get(subject_identifier='B142-040990003-1', schedule_name='c_sec2_schedule1').delete()

OnScheduleCohortCSec.objects.get(subject_identifier='B142-040990002-3', schedule_name='c_sec2_schedule1').delete()

OnScheduleCohortCSec.objects.get(subject_identifier='B142-040990007-2', schedule_name='c_sec1_schedule1').delete()



(1, {'flourish_caregiver.OnScheduleCohortCSec': 1})

In [3]:
from django.apps import apps as django_apps
from edc_base.utils import get_utcnow


def create_consent_version(instance, version):
    consent_version_cls = django_apps.get_model(
        'flourish_caregiver.flourishconsentversion')

    try:
        consent_version_cls.objects.get(
            screening_identifier=instance.screening_identifier)
    except consent_version_cls.DoesNotExist:
        consent_version = consent_version_cls(
            screening_identifier=instance.screening_identifier,
            version=version,
            user_created=instance.user_modified or instance.user_created,
            created=get_utcnow())
        consent_version.save()


cfv = FlourishConsentVersion.objects.values_list('screening_identifier')

missing = SubjectConsent.objects.filter(version='2').exclude(screening_identifier__in=cfv)

for consent in missing:
    create_consent_version(consent, consent.version)

In [4]:
###########Start Here
# Imports
import os 
import pandas as pd
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from datetime import date
from edc_appointment.models import Appointment

from dateutil import parser
from edc_appointment.models import Appointment

pids = []
pids1 = []
#OnScheduleCohortBSec
# OnScheduleCohortCEnrollment
schedules = [OnScheduleCohortASec, OnScheduleCohortBSec, OnScheduleCohortCSec,
            OnScheduleCohortAEnrollment, OnScheduleCohortBEnrollment, OnScheduleCohortCEnrollment]


for s in schedules:
    for a in s.objects.all():
        sn = a.schedule_name

        ccc = CaregiverChildConsent.objects.filter(subject_identifier=a.child_subject_identifier)
        for c in ccc:
            if '2' in sn and c.caregiver_visit_count != 2:
                pids.append(c.subject_identifier)
                c.caregiver_visit_count = 2
                c.save_base(raw=True)
            elif '2' not in sn and c.caregiver_visit_count != 1:
                pids1.append(c.subject_identifier)
                c.caregiver_visit_count = 1
                c.save_base(raw=True)

In [5]:
###########Start Here
# Imports
import os 
import pandas as pd
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from datetime import date
from edc_appointment.models import Appointment

from dateutil import parser
from edc_appointment.models import Appointment
no_appt = []
no_cpe = []
mismatch = []
failed = []
failed_appts = []

enrol_dict = [OnScheduleCohortAEnrollment, OnScheduleCohortBEnrollment, OnScheduleCohortCEnrollment,
             OnScheduleCohortASec, OnScheduleCohortBSec, OnScheduleCohortCSec]

#Traverse all enrollment schedules
for g in enrol_dict:
    
    schedule_objs = g.objects.all()
    
    #check if onschedule datetime is correct, else build dict
    for k in schedule_objs:
        try:
            cpe = CaregiverPreviouslyEnrolled.objects.get(subject_identifier=k.subject_identifier)
        except CaregiverPreviouslyEnrolled.DoesNotExist:
            no_cpe.append(k.subject_identifier)
        
        schedule_name = k.schedule_name
            
        if k.onschedule_datetime.date() != cpe.report_datetime.date():
            print(k, k.subject_identifier, schedule_name, k.schedule_name, k.child_subject_identifier)
            mismatch.append(k.subject_identifier)
            mv = None
            appt = None
            try:
                appt = Appointment.objects.get(subject_identifier=k.subject_identifier,
                                               visit_code='2000M', visit_code_sequence=0,
                                               schedule_name=schedule_name)
            except Appointment.DoesNotExist:
                failed_appts.append(k.subject_identifier)
            else:
                try:
                    mv = MaternalVisit.objects.get(subject_identifier=k.subject_identifier,
                                           visit_code='2000M', visit_code_sequence=0,
                                                   schedule_name=k.schedule_name)
                except MaternalVisit.DoesNotExist:
                    pass
              
                appt.schedule_name = appt.schedule_name + 'x'
                appt.save()
            k.delete()
            
            try:
                cpe.save()
            except:
                failed.append(k.subject_identifier)
            else:
                try:
                    apptm = Appointment.objects.get(subject_identifier=k.subject_identifier,
                                                   visit_code='2000M', visit_code_sequence=0,
                                                   schedule_name=schedule_name)
                except:
                    apptm = None
                    print('oops')

                if mv and apptm:
                    mv.appointment = apptm
                    mv.save_base(raw=True)
                if appt:
                    try:
                        appt.delete()
                    except:
                        failed_appts.append(appt.subject_identifier, appt.schedule_name)
                    

B142-040990504-8 2021-11-25 B142-040990504-8 a_enrol1_schedule1 a_enrol1_schedule1 B142-040990504-8-10
B142-040990581-6 2022-02-09 B142-040990581-6 a_enrol1_schedule1 a_enrol1_schedule1 B142-040990581-6-10
B142-040990006-4 2021-05-06 B142-040990006-4 b_enrol1_schedule1 b_enrol1_schedule1 B142-040990006-4-10
B142-040990009-8 2021-05-07 B142-040990009-8 b_enrol1_schedule1 b_enrol1_schedule1 B142-040990009-8-10
B142-040990044-5 2021-07-02 B142-040990044-5 b_enrol1_schedule1 b_enrol1_schedule1 B142-040990044-5-60
B142-040990585-7 2022-02-11 B142-040990585-7 b_enrol1_schedule1 b_enrol1_schedule1 B142-040990585-7-10
B142-040990008-0 2021-05-07 B142-040990008-0 c_enrol1_schedule1 c_enrol1_schedule1 B142-040990008-0-10
B142-040990623-6 2022-02-08 B142-040990623-6 c_enrol1_schedule1 c_enrol1_schedule1 B142-040990623-6-10
B142-040990023-9 2021-05-17 B142-040990023-9 b_sec1_schedule1 b_sec1_schedule1 B142-040990023-9-10
B142-040990602-0 2022-03-07 B142-040990602-0 b_sec1_schedule1 b_sec1_schedule

In [6]:
no_appt

[]

In [7]:
no_cpe

[]

In [8]:
mismatch

['B142-040990504-8',
 'B142-040990581-6',
 'B142-040990006-4',
 'B142-040990009-8',
 'B142-040990044-5',
 'B142-040990585-7',
 'B142-040990008-0',
 'B142-040990623-6',
 'B142-040990023-9',
 'B142-040990602-0',
 'B142-040990263-1',
 'B142-040990612-9',
 'B142-040990001-5',
 'B142-040990003-1',
 'B142-040990005-6',
 'B142-040990182-3',
 'B142-040990002-3',
 'B142-040990606-1']

In [9]:
failed

[]

In [10]:
failed_appts

[]